In [9]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from sklearn.model_selection import train_test_split
# 4. Model Training


In [10]:
import pandas as pd
mobile = pd.read_csv('Data/mobile_cleaned.csv')
# mobile.drop(columns='time',inplace=True)
mobile.drop_duplicates(inplace=True)
mobile = mobile.reset_index(drop=True)
mobile.reset_index(drop=True, inplace=True)

print(mobile.mobile_strength.min())
print(mobile.mobile_strength.max())
# we have max and min
# we set a range interval based score for the strength
mobile.mobile_strength = mobile.mobile_strength.div(10)
mobile

13
62


,time,lat,long,mobile_strength
0,1712870250,40.112640,-88.218734,3.6
1,1712870260,40.112646,-88.218679,3.6
2,1712870292,40.112550,-88.218558,2.9
3,1712870292,40.112550,-88.218558,3.9
4,1712870300,40.112630,-88.218658,2.9
...,...,...,...,...
2648,1714183561,40.108236,-88.235220,4.8
2649,1714183561,40.108236,-88.235220,5.0
2650,1714183561,40.108236,-88.235220,4.4
2651,1714183561,40.108236,-88.235220,4.7


In [11]:
from sklearn.model_selection import train_test_split
X = mobile[['time', 'lat', 'long']]  # Features: time, latitude, longitude
y = mobile['mobile_strength']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.001, random_state=42)


In [16]:
model = RandomForestRegressor()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)


df = pd.DataFrame({'lat':X_test.lat,'long':X_test.long,'expected':y_test,'predicted':y_pred}).reset_index(drop=True)
df

,lat,long,expected,predicted
0,40.103832,-88.234731,2.4,2.404217
1,40.112556,-88.218597,3.0,2.999500
2,40.110484,-88.219126,3.0,3.000000


In [17]:
print(X_test)

            time        lat       long
2470  1714173002  40.103832 -88.234731
173   1713217264  40.112556 -88.218597
1229  1713228748  40.110484 -88.219126


In [33]:
from sklearn.neighbors import KDTree
higher_signal_points = mobile[mobile.mobile_strength>=df.predicted[0]+1].reset_index(drop=True)[['lat','long']]
print(higher_signal_points)
kd_tree = KDTree(higher_signal_points)

given_point = [[df.lat[0], df.long[0]]]

distance, nearest_index = kd_tree.query(given_point, k=1)

print(distance[0], nearest_index[0])

nearest_point = higher_signal_points.iloc[nearest_index[0][0]]
print(nearest_point)

           lat       long
0    40.112640 -88.218734
1    40.112646 -88.218679
2    40.112550 -88.218558
3    40.112558 -88.218577
4    40.112584 -88.218498
..         ...        ...
455  40.108236 -88.235220
456  40.108236 -88.235220
457  40.108236 -88.235220
458  40.108236 -88.235220
459  40.108236 -88.235220

[460 rows x 2 columns]
[0.00169947] [357]
lat     40.105495
long   -88.235079
Name: 357, dtype: float64
